# Kmeans over a set of GeoTiffs

This notebook loads a set of GeoTiffs into a **RDD** of Tiles, with each Tile being a band in the GeoTiff. Each GeoTiff file contains **SpringIndex-** or **LastFreeze-** value for one year over the entire USA.

Kmeans takes years as dimensions. Hence, the matrix has cells as rows and the years as columns. To cluster on all years, the matrix needs to be transposed. The notebook has two flavors of matrix transpose, locally by the Spark-driver or distributed using the Spark-workers. Once transposed the matrix is converted to a **RDD** of dense vectors to be used by **Kmeans** algorithm from **Spark-MLlib**. The end result is a grid where each cell has a cluster ID which is then saved into a SingleBand GeoTiff. By saving the result into a GeoTiff, the reader can plot it using a Python notebook as the one defined in the [python examples](../examples/python).

<span style="color:red">In this notebook the reader only needs to modify the variables in **Mode of Operation Setup**</span>.

## Dependencies

In [1]:
import sys.process._

import java.io.{ByteArrayInputStream, ByteArrayOutputStream, ObjectInputStream, ObjectOutputStream}


import geotrellis.proj4.CRS
import geotrellis.raster.{CellType, ArrayTile, DoubleArrayTile, Tile, UByteCellType}
import geotrellis.raster.io.geotiff._
import geotrellis.raster.io.geotiff.writer.GeoTiffWriter
import geotrellis.raster.io.geotiff.{GeoTiff, SinglebandGeoTiff}
import geotrellis.spark.io.hadoop._
import org.apache.hadoop.io._
import geotrellis.vector.{Extent, ProjectedExtent}
import org.apache.spark.broadcast.Broadcast
import org.apache.spark.mllib.clustering.{KMeans, KMeansModel}
import org.apache.spark.mllib.linalg.distributed.{CoordinateMatrix, MatrixEntry, RowMatrix}
import org.apache.spark.mllib.linalg.{Vector, Vectors}
import org.apache.spark.rdd.RDD
import org.apache.spark.{SparkConf, SparkContext}

import org.apache.hadoop.io.{IOUtils, SequenceFile}
import org.apache.hadoop.io.SequenceFile.Writer

//Spire is a numeric library for Scala which is intended to be generic, fast, and precise.
import spire.syntax.cfor._

...


## Mode of operation

Here the user can define the mode of operation.
* **rdd_offline_mode**: If false it means the notebook will create all data from scratch and store protected_extent and num_cols_rows into HDFS. Otherwise, these data structures are read from HDFS.
* **matrix_offline_mode**: If false it means the notebook will create a mtrix,  transposed it and save it to HDFS. Otherwise, these data structures are read from HDFS.
* **kmeans_offline_mode**: If false it means the notebook will train kmeans and run kemans and store kmeans model into HDFS. Otherwise, these data structures are read from HDFS.

It is also possible to define which directory of GeoTiffs is to be used and on which **band** to run Kmeans. The options are
* **BloomFinal** or **LeafFinal** which are multi-band (**4 bands**)
* **DamageIndex** and **LastFreeze** which are single-band and if set band_num higher, it will reset to 0

For kmeans the user can define the **number of iterations** and **number of clusters** as an inclusive range. Such range is defined using **minClusters**, **maxClusters**, and **stepClusters**. These variables will set a loop starting at **minClusters** and stopping at **maxClusters** (inclusive), iterating **stepClusters** at the time. <span style="color:red">Note that when using a range **kemans offline mode** is not possible and it will be reset to **online mode**</span>.

### Mode of Operation setup
<a id='mode_of_operation_setup'></a>

In [2]:
//Operation mode
var rdd_offline_mode = true
var matrix_offline_mode = true
var kmeans_offline_mode = true

//GeoTiffs to be read from "hdfs:///user/hadoop/spring-index/"
var dir_path = "hdfs:///user/hadoop/spring-index/"
var offline_dir_path = "hdfs:///user/emma/spring-index/"
var geoTiff_dir = "BloomFinal"
var band_num = 4

//Mask
val toBeMasked = true
val mask_path = "hdfs:///user/hadoop/usa_mask.tif"

//Kmeans number of iterations and clusters
var numIterations = 35
var minClusters = 3
var maxClusters = 16
var stepClusters = 1
var save_kmeans_model = false

rdd_offline_mode = true
matrix_offline_mode = true
kmeans_offline_mode = true
dir_path = hdfs:///user/hadoop/spring-index/
offline_dir_path = hdfs:///user/emma/spring-index/
geoTiff_dir = BloomFinal
band_num = 4
toBeMasked = true
mask_path = hdfs:///user/hadoop/usa_mask.tif
numIterations = 35
minClusters = 3
maxClusters = 16
stepClusters = 1
save_kmeans_model = false


false


<span style="color:red">DON'T MODIFY ANY PIECE OF CODE FROM HERE ON!!!</span>.


### Mode of operation validation

In [3]:
//Validation, do not modify these lines.
var single_band = false
if (geoTiff_dir == "BloomFinal" || geoTiff_dir == "LeafFinal") {
    single_band = false
} else if (geoTiff_dir == "LastFreeze" || geoTiff_dir == "DamageIndex") {
    single_band = true
    if (band_num > 0) {
        println("Since LastFreezze and DamageIndex are single band, we will use band 0!!!")
        band_num  = 0
    }
} else {
    println("Directory unknown, please set either BloomFinal, LeafFinal, LastFreeze or DamageIndex!!!")
}

if (minClusters > maxClusters) {
    maxClusters = minClusters
    stepClusters = 1
}
if (stepClusters < 1) {
    stepClusters = 1
}

//Paths to store data structures for Offline runs
var mask_str = ""
if (toBeMasked)
    mask_str = "_mask"
var grids_noNaN_path = offline_dir_path + geoTiff_dir + "/grids_noNaN" + mask_str
var metadata_path = offline_dir_path + geoTiff_dir + "/metadata" + mask_str
var grids_matrix_path = offline_dir_path + geoTiff_dir + "/grids_matrix" + mask_str

//Check offline modes
var conf = sc.hadoopConfiguration
var fs = org.apache.hadoop.fs.FileSystem.get(conf)

val rdd_offline_exists = fs.exists(new org.apache.hadoop.fs.Path(grids_noNaN_path))
val matrix_offline_exists = fs.exists(new org.apache.hadoop.fs.Path(grids_matrix_path))
                                      
if (rdd_offline_mode != rdd_offline_exists) {
    println("\"Load GeoTiffs\" offline mode is not set properly, i.e., either it was set to false and the required file does not exist or vice-versa. We will reset it to " + rdd_offline_exists.toString())
    rdd_offline_mode = rdd_offline_exists
} 
if (matrix_offline_mode != matrix_offline_exists) {
    println("\"Matrix\" offline mode is not set properly, i.e., either it was set to false and the required file does not exist or vice-versa. We will reset it to " + matrix_offline_exists.toString())
    matrix_offline_mode = matrix_offline_exists
}

if (!fs.exists(new org.apache.hadoop.fs.Path(mask_path))) {
    println("The mask path: " + mask_path + " is invalid!!!")
}

var num_kmeans :Int  = 1
if (minClusters != maxClusters) {
    num_kmeans = ((maxClusters - minClusters) / stepClusters) + 1
}
println(num_kmeans)
var kmeans_model_paths :Array[String] = Array.fill[String](num_kmeans)("")
var wssse_path :String = offline_dir_path + geoTiff_dir + "/wssse"
var geotiff_hdfs_paths :Array[String] = Array.fill[String](num_kmeans)("")
var geotiff_tmp_paths :Array[String] = Array.fill[String](num_kmeans)("")

if (num_kmeans > 1) {
    var numClusters_id = 0
    cfor(minClusters)(_ <= maxClusters, _ + stepClusters) { numClusters =>
        kmeans_model_paths(numClusters_id) = offline_dir_path + geoTiff_dir + "/kmeans_model_" + numClusters + "_" + numIterations
        
        //Check if the file exists
        val kmeans_exist = fs.exists(new org.apache.hadoop.fs.Path(kmeans_model_paths(numClusters_id)))
        if (kmeans_exist && !kmeans_offline_mode) {
            println("The kmeans model path " + kmeans_model_paths(numClusters_id) + " exists, please remove it.")
        } else if (!kmeans_exist && kmeans_offline_mode) {
            kmeans_offline_mode = false
        }
        
        geotiff_hdfs_paths(numClusters_id) = offline_dir_path + geoTiff_dir + "/clusters_" + numClusters + "_" + numIterations + ".tif"
        geotiff_tmp_paths(numClusters_id) = "/tmp/clusters_" + geoTiff_dir + "_" + numClusters + "_" + numIterations + ".tif"
        if (fs.exists(new org.apache.hadoop.fs.Path(geotiff_hdfs_paths(numClusters_id)))) {
            println("There is already a GeoTiff with the path: " + geotiff_hdfs_paths(numClusters_id) + ". Please make either a copy or move it to another location, otherwise, it will be over-written.")
        }
        numClusters_id += 1
    }
    kmeans_offline_mode = false
} else { 
    kmeans_model_paths(0) = offline_dir_path + geoTiff_dir + "/kmeans_model_" + minClusters + "_" + numIterations
    val kmeans_offline_exists = fs.exists(new org.apache.hadoop.fs.Path(kmeans_model_paths(0)))
    if (kmeans_offline_mode != kmeans_offline_exists) {
        println("\"Kmeans\" offline mode is not set properly, i.e., either it was set to false and the required file does not exist or vice-versa. We will reset it to " + kmeans_offline_exists.toString())
        kmeans_offline_mode = kmeans_offline_exists
    }
    geotiff_hdfs_paths(0) = offline_dir_path + geoTiff_dir + "/clusters_" + minClusters + "_" + numIterations + ".tif"
    geotiff_tmp_paths(0) = "/tmp/clusters_" + geoTiff_dir + "_" + minClusters + "_" + numIterations + ".tif"
    if (fs.exists(new org.apache.hadoop.fs.Path(geotiff_hdfs_paths(0)))) {
        println("There is already a GeoTiff with the path: " + geotiff_hdfs_paths(0) + ". Please make either a copy or move it to another location, otherwise, it will be over-written.")
    }
}

Waiting for a Spark session to start...

"Load GeoTiffs" offline mode is not set properly, i.e., either it was set to false and the required file does not exist or vice-versa. We will reset it to false
"Matrix" offline mode is not set properly, i.e., either it was set to false and the required file does not exist or vice-versa. We will reset it to false
14
There is already a GeoTiff with the path: hdfs:///user/emma/spring-index/BloomFinal/clusters_3_35.tif. Please make either a copy or move it to another location, otherwise, it will be over-written.
There is already a GeoTiff with the path: hdfs:///user/emma/spring-index/BloomFinal/clusters_4_35.tif. Please make either a copy or move it to another location, otherwise, it will be over-written.
There is already a GeoTiff with the path: hdfs:///user/emma/spring-index/BloomFinal/clusters_5_35.tif. Please make either a copy or move it to another location, otherwise, it will be over-written.
There is already a GeoTiff with the path: hdfs:///user/emma/spring-index/BloomFinal/cluster

single_band = false
mask_str = _mask
grids_noNaN_path = hdfs:///user/emma/spring-index/BloomFinal/grids_noNaN_mask
metadata_path = hdfs:///user/emma/spring-index/BloomFinal/metadata_mask
grids_matrix_path = hdfs:///user/emma/spring-index/BloomFinal/grids_matrix_mask
conf = Configuration: core-default.xml, core-site.xml, mapred-default.xml, mapred-site.xml, yarn-default.xml, yarn-site.xml, hdfs-default.xml, hdfs-site.xml, file:/usr/lib/spark-2.1.1-bin-without-hadoop/conf/hive-site.xml
fs = DFS[DFSClient[clientName=DFSClient_NONMAPREDUCE_934661335_39, ugi=emma (auth:SIMPLE)]]
rdd_offline_exists = false
matrix_offline_exists = false
num_kmeans = 14


kmeans_m...


14

## Functions to (de)serialize any structure into Array[Byte]

In [4]:
def serialize(value: Any): Array[Byte] = {
    val out_stream: ByteArrayOutputStream = new ByteArrayOutputStream()
    val obj_out_stream = new ObjectOutputStream(out_stream)
    obj_out_stream.writeObject(value)
    obj_out_stream.close
    out_stream.toByteArray
}

def deserialize(bytes: Array[Byte]): Any = {
    val obj_in_stream = new ObjectInputStream(new ByteArrayInputStream(bytes))
    val value = obj_in_stream.readObject
    obj_in_stream.close
    value
}

serialize: (value: Any)Array[Byte]
deserialize: (bytes: Array[Byte])Any


## Load GeoTiffs

Using GeoTrellis all GeoTiffs of a directory will be loaded into a RDD. Using the RDD, we extract a grid from the first file to lated store the Kmeans cluster_IDS, we build an Index for populate such grid and we convert all NaN values to **-1**.

In [5]:
val t0 = System.nanoTime()
//Global variables
var projected_extent = new ProjectedExtent(new Extent(0,0,0,0), CRS.fromName("EPSG:3857"))
var grids_noNaN_RDD: RDD[Array[Double]] = sc.emptyRDD
var num_cols_rows :(Int, Int) = (0, 0)
var cellT :CellType = UByteCellType
var mask_tile0 :Tile = new SinglebandGeoTiff(geotrellis.raster.ArrayTile.empty(cellT, num_cols_rows._1, num_cols_rows._2), projected_extent.extent, projected_extent.crs, Tags.empty, GeoTiffOptions.DEFAULT).tile

//Load Mask
if (toBeMasked) {
    val mask_tiles_RDD = sc.hadoopGeoTiffRDD(mask_path).values
    val mask_tiles_withIndex = mask_tiles_RDD.zipWithIndex().map{case (e,v) => (v,e)}
    mask_tile0 = (mask_tiles_withIndex.filter(m => m._1==0).filter(m => !m._1.isNaN).values.collect())(0)
}

//Local variables
val pattern: String = "tif"
val filepath: String = dir_path + geoTiff_dir

if (rdd_offline_mode) {
    grids_noNaN_RDD = sc.objectFile(grids_noNaN_path)

    val metadata = sc.sequenceFile(metadata_path, classOf[IntWritable], classOf[BytesWritable]).map(_._2.copyBytes()).collect()
    projected_extent = deserialize(metadata(0)).asInstanceOf[ProjectedExtent]
    num_cols_rows = (deserialize(metadata(1)).asInstanceOf[Int], deserialize(metadata(2)).asInstanceOf[Int])
} else {
    if (single_band) {
        //Lets load a Singleband GeoTiffs and return RDD just with the tiles.
        val geos_RDD = sc.hadoopGeoTiffRDD(filepath, pattern)
        val tiles_RDD = geos_RDD.values
            
        //Retrive the numbre of cols and rows of the Tile's grid
        val tiles_withIndex = tiles_RDD.zipWithIndex().map{case (e,v) => (v,e)}
        val tile0 = (tiles_withIndex.filter(m => m._1==0).values.collect())(0)
        num_cols_rows = (tile0.cols,tile0.rows)
        cellT = tile0.cellType
    
        //Retrieve the ProjectExtent which contains metadata such as CRS and bounding box
        val projected_extents_withIndex = geos_RDD.keys.zipWithIndex().map{case (e,v) => (v,e)}
        projected_extent = (projected_extents_withIndex.filter(m => m._1 == 0).values.collect())(0)
        
        if (toBeMasked) {
            val mask_tile_broad :Broadcast[Tile] = sc.broadcast(mask_tile0)
            grids_noNaN_RDD = tiles_RDD.map(m => m.localInverseMask(mask_tile_broad.value, 1, 0).toArrayDouble().map(m => if ((m == 0.0) || (m.isNaN)) -1.0 else m))
        } else {
            grids_noNaN_RDD = tiles_RDD.map(m => m.toArrayDouble().map(m => if (m.isNaN) -1.0 else m))
        }
    } else {
        //Lets load Multiband GeoTiffs and return RDD just with the tiles.
        val geos_RDD = sc.hadoopMultibandGeoTiffRDD(filepath, pattern)
        val tiles_RDD = geos_RDD.values
            
        //Retrive the numbre of cols and rows of the Tile's grid
        val tiles_withIndex = tiles_RDD.zipWithIndex().map{case (e,v) => (v,e)}
        val tile0 = (tiles_withIndex.filter(m => m._1==0).values.collect())(0)

        num_cols_rows = (tile0.cols,tile0.rows)
        cellT = tile0.cellType
    
        //Retrieve the ProjectExtent which contains metadata such as CRS and bounding box
        val projected_extents_withIndex = geos_RDD.keys.zipWithIndex().map{case (e,v) => (v,e)}
        projected_extent = (projected_extents_withIndex.filter(m => m._1 == 0).values.collect())(0)
    
        //Lets read the average of the Spring-Index which is stored in the 4th band
        if (toBeMasked) {
            val mask_tile_broad :Broadcast[Tile] = sc.broadcast(mask_tile0)
            grids_noNaN_RDD = tiles_RDD.map(m => m.band(3).localInverseMask(mask_tile_broad.value, 1, 0).toArrayDouble().map(m => if ((m == 0.0) || (m.isNaN)) -1.0 else m))
        } else {
            grids_noNaN_RDD = tiles_RDD.map(m => m.band(3).toArrayDouble().map(m => if (m.isNaN) -1.0 else m))
        }
    }  
    //Store data in HDFS
    grids_noNaN_RDD.saveAsObjectFile(grids_noNaN_path)
    
    val writer: SequenceFile.Writer = SequenceFile.createWriter(conf,
        Writer.file(metadata_path),
        Writer.keyClass(classOf[IntWritable]),
        Writer.valueClass(classOf[BytesWritable])
    )

    writer.append(new IntWritable(1), new BytesWritable(serialize(projected_extent)))
    writer.append(new IntWritable(2), new BytesWritable(serialize(num_cols_rows._1)))
    writer.append(new IntWritable(3), new BytesWritable(serialize(num_cols_rows._2)))
    writer.hflush()
    writer.close()
}
val t1 = System.nanoTime()
println("Elapsed time: " + (t1 - t0) + "ns")

[Stage 5:=======================================================> (35 + 1) / 36]Elapsed time: 720700223515ns


t0 = 116568168752299
projected_extent = ProjectedExtent(Extent(-126.30312894720473, 14.29219617034159, -56.162671563152486, 49.25462702827337),geotrellis.proj4.CRS$$anon$3@41d0d1b7)
grids_noNaN_RDD = MapPartitionsRDD[21] at map at <console>:138
num_cols_rows = (7808,3892)
cellT = float64raw
mask_tile0 = FloatRawArrayTile([F@c44c716,7808,3892)
pattern = tif
filepath = hdfs:///user/hadoop/spring-index/BloomFinal
t1 = 117288868975814


117288868975814

## Matrix

We need to do a Matrix transpose to have clusters per cell and not per year. With a GeoTiff representing a single year, the loaded data looks liks this:
```
bands_RDD.map(s => Vectors.dense(s)).cache()

//The vectors are rows and therefore the matrix will look like this:
[
Vectors.dense(0.0, 1.0, 2.0),
Vectors.dense(3.0, 4.0, 5.0),
Vectors.dense(6.0, 7.0, 8.0),
Vectors.dense(9.0, 0.0, 1.0)
]
```

To achieve that we convert the **RDD[Vector]** into a distributed Matrix, a [**CoordinateMatrix**](https://spark.apache.org/docs/latest/mllib-data-types.html#coordinatematrix), which as a **transpose** method.

In [6]:
val t0 = System.nanoTime()
//Global variables
var grids_matrix: RDD[Vector] = sc.emptyRDD

if (matrix_offline_mode) {
    grids_matrix = sc.objectFile(grids_matrix_path)
} else {
    val mat :RowMatrix = new RowMatrix(grids_noNaN_RDD.map(m => Vectors.dense(m)))

    // Split the matrix into one number per line.
    val byColumnAndRow = mat.rows.zipWithIndex.map {
        case (row, rowIndex) => row.toArray.zipWithIndex.map {
            case (number, columnIndex) => new MatrixEntry(rowIndex, columnIndex, number)
        }   
    }.flatMap(x => x)
    
    val matt: CoordinateMatrix = new CoordinateMatrix(byColumnAndRow)
    val matt_T = matt.transpose()
    //grids_matrix = matt_T.toRowMatrix().rows
    grids_matrix = matt_T.toIndexedRowMatrix().rows.sortBy(_.index).map(_.vector)
    grids_matrix.saveAsObjectFile(grids_matrix_path)
}
val t1 = System.nanoTime()
println("Elapsed time: " + (t1 - t0) + "ns")

[Stage 12:======================================================> (31 + 1) / 32]Elapsed time: 1628018879622ns


t0 = 117294627315186
grids_matrix = MapPartitionsRDD[39] at map at <console>:86
t1 = 118922646194808


118922646194808

## Kmeans

We use Kmeans from Sparl-MLlib. The user should only modify the variables on Kmeans setup.

### Kmeans Training

In [7]:
val t0 = System.nanoTime()
//Global variables
var kmeans_models :Array[KMeansModel] = new Array[KMeansModel](num_kmeans)
var wssse_data :List[(Int, Int, Double)] = List.empty

if (kmeans_offline_mode) {
    var numClusters_id = 0
    cfor(minClusters)(_ <= maxClusters, _ + stepClusters) { numClusters =>
        if (!fs.exists(new org.apache.hadoop.fs.Path(kmeans_model_paths(numClusters_id)))) {
            println("One of the files does not exist, we will abort!!!")
            System.exit(0)
        } else {
            kmeans_models(numClusters_id) = KMeansModel.load(sc, kmeans_model_paths(numClusters_id))
        }
        numClusters_id += 1
    }
    val wssse_data_RDD :RDD[(Int, Int, Double)]  = sc.objectFile(wssse_path)
    wssse_data  = wssse_data_RDD.collect().toList
} else {
    var numClusters_id = 0
    if (fs.exists(new org.apache.hadoop.fs.Path(wssse_path))) {
        val wssse_data_RDD :RDD[(Int, Int, Double)]  = sc.objectFile(wssse_path)
        wssse_data  = wssse_data_RDD.collect().toList
    }
    grids_matrix.cache()
    cfor(minClusters)(_ <= maxClusters, _ + stepClusters) { numClusters =>
        println(numClusters)
        kmeans_models(numClusters_id) = {
            KMeans.train(grids_matrix, numClusters, numIterations)
        }

        // Evaluate clustering by computing Within Set Sum of Squared Errors
        val WSSSE = kmeans_models(numClusters_id).computeCost(grids_matrix)
        println("Within Set Sum of Squared Errors = " + WSSSE)
                
        wssse_data = wssse_data :+ (numClusters, numIterations, WSSSE)
        
        //Save kmeans model
        if (save_kmeans_model) {
            if (!fs.exists(new org.apache.hadoop.fs.Path(kmeans_model_paths(numClusters_id)))) {
                kmeans_models(numClusters_id).save(sc, kmeans_model_paths(numClusters_id))
            }
        }
        numClusters_id += 1
    }

    //Un-persist it to save memory
    grids_matrix.unpersist()
    
    if (fs.exists(new org.apache.hadoop.fs.Path(wssse_path))) {
        println("We will delete the wssse file")
        try { fs.delete(new org.apache.hadoop.fs.Path(wssse_path), true) } catch { case _ : Throwable => { } }
    }
    
    println("Lets create it with the new data")
    sc.parallelize(wssse_data, 1).saveAsObjectFile(wssse_path)
}
val t1 = System.nanoTime()
println("Elapsed time: " + (t1 - t0) + "ns")

3
Within Set Sum of Squared Errors = 1.2960778421007756E11                        
4
Within Set Sum of Squared Errors = 7.512159562280254E10                         
5
Within Set Sum of Squared Errors = 5.123675278927168E10                         
6
Within Set Sum of Squared Errors = 3.846708033994191E10                         
7
Within Set Sum of Squared Errors = 3.105469424051701E10                         
8
Within Set Sum of Squared Errors = 2.657219149361745E10                         
9
Within Set Sum of Squared Errors = 2.3102424425223885E10                        
10
Within Set Sum of Squared Errors = 2.0896273140021423E10                        
11
Within Set Sum of Squared Errors = 1.935491109936614E10                         
12
Within Set Sum of Squared Errors = 1.8039011596280605E10                        
13
Within Set Sum of Squared Errors = 1.7999790559466667E10                        
14
Within Set Sum of Squared Errors = 1.63046263164963E10                          

t0 = 118941683543930
kmeans_models = Array(org.apache.spark.mllib.clustering.KMeansModel@5f6f6941, org.apache.spark.mllib.clustering.KMeansModel@73e7d1ee, org.apache.spark.mllib.clustering.KMeansModel@7ef9f3e2, org.apache.spark.mllib.clustering.KMeansModel@5f941c34, org.apache.spark.mllib.clustering.KMeansModel@7b1553e6, org.apache.spark.mllib.clustering.KMeansModel@36b085cb, org.apache.spark.mllib.clustering.KMeansModel@7e7061c, org.apache.spark.mllib.clustering.KMeansModel@786dcf68, org.apache.spark.mllib.clustering.KMeansModel@29b639e0, org.apache.spark.mllib.clustering.KMeansModel@1b9a6fc0, org.apache.spark.mllib.clustering.KMeansModel@24725d49, org.apache.spark.mllib.clustering.KMeansModel@32f2c020, org.apache.spark.mllib....


[org.apache.spark.mllib.clustering.KMeansModel@5f6f6941, org.apache.spark.mllib.clustering.KMeansModel@73e7d1ee, org.apache.spark.mllib.clustering.KMeansModel@7ef9f3e2, org.apache.spark.mllib.clustering.KMeansModel@5f941c34, org.apache.spark.mllib.clustering.KMeansModel@7b1553e6, org.apache.spark.mllib.clustering.KMeansModel@36b085cb, org.apache.spark.mllib.clustering.KMeansModel@7e7061c, org.apache.spark.mllib.clustering.KMeansModel@786dcf68, org.apache.spark.mllib.clustering.KMeansModel@29b639e0, org.apache.spark.mllib.clustering.KMeansModel@1b9a6fc0, org.apache.spark.mllib.clustering.KMeansModel@24725d49, org.apache.spark.mllib.clustering.KMeansModel@32f2c020, org.apache.spark.mllib.clustering.KMeansModel@2a9396a7, org.apache.spark.mllib.clustering.KMeansModel@62bf085a]

### Inspect WSSSE

In [8]:
val t0 = System.nanoTime()
//current
println(wssse_data)

//from disk
if (fs.exists(new org.apache.hadoop.fs.Path(wssse_path))) {
    var wssse_data_tmp :RDD[(Int, Int, Double)] = sc.objectFile(wssse_path)//.collect()//.toList
    println(wssse_data_tmp.collect().toList)    
}
}

Name: Compile Error
Message: <console>:10: error: eof expected but '}' found.
}
^

StackTrace: 

### Run Kmeans clustering

Run Kmeans and obtain the clusters per each cell.

In [9]:
val t0 = System.nanoTime()
//Cache it so kmeans is more efficient
grids_matrix.cache()

var kmeans_res: Array[RDD[Int]] = Array.fill(num_kmeans)(sc.emptyRDD)
var numClusters_id = 0
cfor(minClusters)(_ <= maxClusters, _ + stepClusters) { numClusters =>
    kmeans_res(numClusters_id) = kmeans_models(numClusters_id).predict(grids_matrix)
    numClusters_id += 1
}

//Un-persist it to save memory
grids_matrix.unpersist()
val t1 = System.nanoTime()
println("Elapsed time: " + (t1 - t0) + "ns")

Elapsed time: 142030419ns


t0 = 122162075645709
kmeans_res = Array(MapPartitionsRDD[1257] at map at KMeansModel.scala:69, MapPartitionsRDD[1258] at map at KMeansModel.scala:69, MapPartitionsRDD[1259] at map at KMeansModel.scala:69, MapPartitionsRDD[1260] at map at KMeansModel.scala:69, MapPartitionsRDD[1261] at map at KMeansModel.scala:69, MapPartitionsRDD[1262] at map at KMeansModel.scala:69, MapPartitionsRDD[1263] at map at KMeansModel.scala:69, MapPartitionsRDD[1264] at map at KMeansModel.scala:69, MapPartitionsRDD[1265] at map at KMeansModel.scala:69, MapPartitionsRDD[1266] at map at KMeansModel.scala:69, MapPartitionsRDD[1267] at map at KMeansModel.scala:69, MapPartitionsRDD[1268] at map at KMeansModel.scala:69, MapPartitionsRDD[1269] at map at KMeansModel.scala:69...


[MapPartitionsRDD[1257] at map at KMeansModel.scala:69, MapPartitionsRDD[1258] at map at KMeansModel.scala:69, MapPartitionsRDD[1259] at map at KMeansModel.scala:69, MapPartitionsRDD[1260] at map at KMeansModel.scala:69, MapPartitionsRDD[1261] at map at KMeansModel.scala:69, MapPartitionsRDD[1262] at map at KMeansModel.scala:69, MapPartitionsRDD[1263] at map at KMeansModel.scala:69, MapPartitionsRDD[1264] at map at KMeansModel.scala:69, MapPartitionsRDD[1265] at map at KMeansModel.scala:69, MapPartitionsRDD[1266] at map at KMeansModel.scala:69, MapPartitionsRDD[1267] at map at KMeansModel.scala:69, MapPartitionsRDD[1268] at map at KMeansModel.scala:69, MapPartitionsRDD[1269] at map at KMeansModel.scala:69, MapPartitionsRDD[1270] at map at KMeansModel.scala:69]

#### Sanity test

It can be skipped, it only shows the cluster ID for the first 50 cells

In [10]:
val t0 = System.nanoTime()
val kmeans_res_out = kmeans_res(0).take(150)
kmeans_res_out.foreach(print)

println(kmeans_res_out.size)
val t1 = System.nanoTime()
println("Elapsed time: " + (t1 - t0) + "ns")

111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111150
Elapsed time: 3689349210ns


t0 = 122163379270287
kmeans_res_out = Array(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1)
t1 = 122167068619497


122167068619497

## Build GeoTiff with Kmeans cluster_IDs

The Grid with the cluster IDs is stored in a SingleBand GeoTiff and uploaded to HDFS.

### Assign cluster ID to each grid cell and save the grid as SingleBand GeoTiff

To assign the clusterID to each grid cell it is necessary to get the indices of gird cells they belong to. To join the two RDDS the knowledge was obtaing from a stackoverflow post on [how to perform basic joins of two rdd tables in spark using python](https://stackoverflow.com/questions/31257077/how-do-you-perform-basic-joins-of-two-rdd-tables-in-spark-using-python).

In [11]:
val t0 = System.nanoTime()
var numClusters_id = 0

cfor(minClusters)(_ <= maxClusters, _ + stepClusters) { numClusters =>
    val grid_clusters_res = kmeans_res(numClusters_id)
        
    //Define a Tile
    val cluster_cells :Array[Double] = grid_clusters_res.map(m => m.toDouble).collect()
    val cluster_cellsD = DoubleArrayTile(cluster_cells, num_cols_rows._1, num_cols_rows._2)
    val cluster_tile = geotrellis.raster.DoubleArrayTile.empty(num_cols_rows._1, num_cols_rows._2)
    cfor(0)(_ < num_cols_rows._1, _ + 1) { col =>
        cfor(0)(_ < num_cols_rows._2, _ + 1) { row =>
            val v = cluster_cellsD.getDouble(col, row)
            if (v == -1)
                cluster_tile.setDouble(col, row, Double.NaN)
            else
                cluster_tile.setDouble(col, row, v)
        }
    }

    val geoTif = new SinglebandGeoTiff(cluster_tile, projected_extent.extent, projected_extent.crs, Tags.empty, GeoTiffOptions(compression.DeflateCompression))
    
    //Save to /tmp/
    GeoTiffWriter.write(geoTif, geotiff_tmp_paths(numClusters_id))

    //Upload to HDFS
    var cmd = "hadoop dfs -copyFromLocal -f " + geotiff_tmp_paths(numClusters_id) + " " + geotiff_hdfs_paths(numClusters_id)
    Process(cmd)!
    
    //Remove from /tmp/
    cmd = "rm -fr " + geotiff_tmp_paths(numClusters_id)
    Process(cmd)!
    
    numClusters_id += 1
}
val t1 = System.nanoTime()
println("Elapsed time: " + (t1 - t0) + "ns")

[Stage 2258:====================================================> (31 + 1) / 32]DEPRECATED: Use of this script to execute hdfs command is deprecated.
Instead use the hdfs command for it.

[Stage 2261:====================================================> (31 + 1) / 32]DEPRECATED: Use of this script to execute hdfs command is deprecated.
Instead use the hdfs command for it.

[Stage 2264:====================================================> (31 + 1) / 32]DEPRECATED: Use of this script to execute hdfs command is deprecated.
Instead use the hdfs command for it.

[Stage 2267:==================================================>   (30 + 2) / 32]DEPRECATED: Use of this script to execute hdfs command is deprecated.
Instead use the hdfs command for it.

[Stage 2270:====================================================> (31 + 1) / 32]DEPRECATED: Use of this script to execute hdfs command is deprecated.
Instead use the hdfs command for it.

[Stage 2273:================================================

t0 = 122168532275082
numClusters_id = 14
t1 = 122814129702078


122814129702078

# [Visualize results](../stable/plot_kmeans_clusters.ipynb)